## Intro
In this notebook, we are going to analyse the seismogram outputs for varying moment tensor/structure with respect to each model parameter (mxx,myy,mzz,mxy,mxz,myz,moho-d). By doing this, we can obtain the sensitivity of the seismograms to each of the parameters.

In [17]:
from obspy.taup import TauPyModel
from os.path import join,isfile
from os import listdir
from obspy.geodetics import kilometer2degrees
from SS_MTI import Gradient, PhaseTracer

## Fixed parameters from inversion

In [10]:
src_depth = 20.0
epi_in_km = 1774.7380
epi = kilometer2degrees(epi_in_km, radius=3389.5)
baz = 0.0

dt = 0.025

phases = ["P", "S", "P", "S", "S"]
comps = ["Z", "T", "R", "Z", "R"]
t_pres = [1, 1, 1, 1, 1]
t_posts = [30, 30, 30, 30, 30]
ylims = [1e-9, 1e-9, 2e-9, 3e-9, 2e-9]

fmin = 0.2
fmax = 0.6
zerophase = False

## Epsilon
We will look at an example, where the steps in each model parameter direction is done with an epsilon of 0.01

In [11]:
epsilon = 0.01

## Start model
First, we load the seismograms from our initial model (m0 = [m_rr, m_tt, m_pp, m_rt, m_rp, m_tp, moho-d])

In [13]:
npz_file

['77.368.npz']

In [14]:
m0_path = "/home/nienke/Documents/Research/SS_MTI/External_packages/Test_reflectivity/m0_gradient_descent/Update_1/It_0"
st_m0 = Gradient.read_refl_mseeds(path=m0_path)
# window the data:
npz_file = [f for f in listdir(m0_path) if f.endswith(".npz") if isfile(join(m0_path, f))]
Taup = TauPyModel(join(m0_path,npz_file[0]))
m0_tts = [PhaseTracer.get_traveltime(Taup, phase, src_depth, epi) for phase in phases]
st_m0_w, st_m0_full, s_m0 = Gradient.window(
    st_m0,
    phases,
    comps,
    m0_tts,
    t_pres,
    t_posts,
    fmin,
    fmax,
    zerophase,
)

## Update history
Here, we have the update history when we use epsilon of 0.01. Observe that 7 forward runs are necessary to find the gradient w.r.t the entire model. Now we can now visualize the effect of each update on the waveforms, so for example we can analyse if an update in mxx relates to only amplitude changes and the change in moho only in phase shifts, or is it may be sensible to various changes at the same time?

-- Update History --

$m0$: [0.2,0.8,0.,0.,0.,0.,77.368]

1.$m0 + \epsilon  m_{rr}$: [0.20166667,0.8,0.,0.,0.,0.,77.368] -- 
forward run in iteration: 91 --
misfit = 1037.16085968 

2.$m0 + \epsilon  m_{tt}$: [0.2,0.80166667,0.,0.,0.,0.,77.368] --
forward run in iteration: 92 --
misfit = 1041.56079792 

3.$m0 + \epsilon  m_{pp}$: [0.2,0.8,1.66666667e-03,0.,0.,0.,77.368] --
forward run in iteration: 93 --
misfit = 1041.66531012

4.$m0 + \epsilon  m_{rt}$: [0.2,0.8,0.,1.66666667e-03,0.,0.,77.368] --
forward run in iteration: 94 --
misfit = 1037.16085968

5.$m0 + \epsilon  m_{rp}$: [0.2,0.8,0.,0.,1.66666667e-03,0.,77.368] --
forward run in iteration: 95 --
misfit = 1041.15464715

6.$m0 + \epsilon  m_{tp}$: [0.2,0.8,0.,0.,0.,1.66666667e-03 ,77.368] --
forward run in iteration: 96 --
misfit = 1042.42410446

7.$m0 + \epsilon  m_{moho-d}$: [0.2,0.8,0.,0.,0.,0.,78.14168] --
forward run in iteration: 97 --
misfit = 1360.46697213


## 1. $m0 + \epsilon  m_{rr}$

First, we load the data that belongs to this update

In [15]:
m1_path = "/home/nienke/Documents/Research/SS_MTI/External_packages/Test_reflectivity/m0_gradient_descent/Update_1/It_91"
st_m1 = Gradient.read_refl_mseeds(path=m1_path)
# window the data:
npz_file = [f for f in listdir(m1_path) if f.endswith(".npz") if isfile(join(m1_path, f))]
Taup = TauPyModel(join(m1_path,npz_file[0]))
m1_tts = [PhaseTracer.get_traveltime(Taup, phase, src_depth, epi) for phase in phases]
st_m1_w, st_m1_full, s_m1 = Gradient.window(
    st_m1,
    phases,
    comps,
    m1_tts,
    t_pres,
    t_posts,
    fmin,
    fmax,
    zerophase,
)

Then, we would like to calculate the residual between the two seismograms ($m0$ and $m0 + \epsilon  m_{rr}$)

In [22]:
st_res1 = st_m0_w.copy()
for i, (tr_m0, tr_m1) in enumerate(zip(st_m0_w,st_m1_w)):
    res_data = tr_m0.data - tr_m1.data
    st_res1[i].data = res_data
